# Example of computing vibrational energy levels of a triatomic molecule using the Taylor series expansion approximation for the kinetic and potential energy operators

Compute vibrational states of $\text{H}_2\text{S}$ molecule using the potential energy surface from
[A. A. A. Azzam, J. Tennyson, S. N. Yurchenko, O. V. Naumenko, "ExoMol molecular line lists - XVI. The rotation–vibration spectrum of hot H2S", MNRAS 460, 4063–4074 (2016)](https://doi.org/10.1093/mnras/stw1133)


In [1]:
from typing import Callable, List

import jax
import jax.numpy as jnp
import numpy as np
from numpy.polynomial.hermite import hermder, hermgauss, hermval
from scipy import optimize
from scipy.special import factorial

from vibrojet.keo import Gmat, com
from vibrojet.potentials import h2s_AYT2
from vibrojet.taylor import deriv_list

jax.config.update("jax_enable_x64", True)

Compute equilibrium coordinates, around which the Taylor series expansions will be built

In [2]:
vmin = optimize.minimize(h2s_AYT2.poten, [1, 1, 90 * np.pi / 180])
r0 = vmin.x
v0 = vmin.fun
print("Equilibrium coordinates:", r0)
print("Min of the potential:", v0)

Equilibrium coordinates: [1.3358387  1.3358387  1.61042427]
Min of the potential: -0.0007846164047977397


Define mapping function from internal valence coordinates, $r_1$, $r_2$, and $\alpha$, to Cartesian coordinates

In [3]:
masses = [31.97207070, 1.00782505, 1.00782505]  # masses of S, H, H

ncoo = len(r0)


@com(masses)
def internal_to_cartesian(coords):
    r1, r2, alpha = coords
    return jnp.array(
        [
            [0.0, 0.0, 0.0],
            [r1 * jnp.cos(alpha / 2), 0.0, r1 * jnp.sin(alpha / 2)],
            [r2 * jnp.cos(alpha / 2), 0.0, -r2 * jnp.sin(alpha / 2)],
        ]
    )

Utility function for generating index combinations corresponding to products of primitive basis functions as well as derivative indices used in the expansion of operators

In [4]:
def generate_prod_ind(
    indices: List[List[int]],
    select: Callable[[List[int]], bool] = lambda _: True,
    batch_size: int = None,
):
    no_elem = np.array([len(elem) for elem in indices])
    tot_size = np.prod(no_elem)
    if batch_size is None:
        batch_size = tot_size
    no_batches = (tot_size + batch_size - 1) // batch_size

    for ibatch in range(no_batches):
        start_ind = ibatch * batch_size
        end_ind = np.minimum(start_ind + batch_size, tot_size)
        batch_ind = np.arange(start_ind, end_ind)
        multi_ind = np.array(np.unravel_index(batch_ind, no_elem))
        indices_out = np.array(
            [indices[icoo][multi_ind[icoo, :]] for icoo in range(len(indices))]
        ).T
        select_ind = np.where(np.asarray([select(elem) for elem in indices_out]))
        yield indices_out[select_ind], multi_ind[:, select_ind[0]]

Compute Taylor series expansion of the $G$-matrix and potential

$$
G_{\lambda,\mu}=\sum_{\mathbf{t}} g_{\mathbf{t},\lambda,\mu} (r_1-r_1^\text{(eq)})^{t_1}(r_2-r_2^\text{(eq)})^{t_2}(\alpha-\alpha^\text{(eq)})^{t_3},
$$

$$
V=\sum_{\mathbf{t}} f_{\mathbf{t}} (r_1-r_1^\text{(eq)})^{t_1}(r_2-r_2^\text{(eq)})^{t_2}(\alpha-\alpha^\text{(eq)})^{t_3},
$$

 where the derivative multi-index $\mathbf{t}$ is stored in `deriv_ind`, and the expansion coefficients $g_{\mathbf{t},\lambda,\mu}$ and $f_{\mathbf{t}}$ in `Gmat_coefs[:len(deriv_ind),:3N, :3N]` and `poten_coefs[:len(deriv_ind)]` respectively, where $N$ is the number of atoms

In [5]:
max_pow = 6
powers = [np.arange(max_pow + 1)] * ncoo
deriv_ind, deriv_mind = next(
    generate_prod_ind(powers, select=lambda ind: np.sum(ind) <= max_pow)
)

print("Max expansion power:", max_pow)
print("Number of expansion terms:", len(deriv_ind))

Max expansion power: 6
Number of expansion terms: 84


In [6]:
print("compute expansion of potential ...")
poten_coefs = deriv_list(h2s_AYT2.poten, deriv_ind, r0, if_taylor=True)

compute expansion of potential ...


In [7]:
print("compute expansion of G-matrix ...")
Gmat_coefs = deriv_list(
    lambda x: Gmat(x, masses, internal_to_cartesian), deriv_ind, r0, if_taylor=True
)

compute expansion of G-matrix ...


Define primitive Hermite basis functions $\mathcal{H}_n(x)$ and their derivatives $d\mathcal{H}_n(x)/dx$

In [8]:
def hermite(x, n):
    sqsqpi = np.sqrt(np.sqrt(np.pi))
    c = np.diag(1.0 / np.sqrt(2.0**n * factorial(n)) / sqsqpi)
    f = hermval(np.asarray(x), c) * np.exp(-(x**2) / 2)
    return f.T

def hermite_deriv(x, n):
    sqsqpi = np.sqrt(np.sqrt(np.pi))
    c = np.diag(1.0 / np.sqrt(2.0**n * factorial(n)) / sqsqpi)
    h = hermval(np.asarray(x), c)
    dh = hermval(np.asarray(x), hermder(c, m=1))
    f = (dh - h * x) * np.exp(-(x**2) / 2)
    return f.T

Define mapping between the coordinate $x\in (-\infty, \infty)$ of Hermite basis functions and the internal valence coordinates as $r_1=a_1 x_1 + b_1$, $r_2=a_2x_2+b_2$, $\alpha=a_3x_3+b_3$. The parameters $a_1,b_1,...,b_3$ are determined by mapping the vibrational Hamiltonian in valence coordinates onto the harmonic-oscillator Hamiltonian.

In [9]:
mask = deriv_ind != 0

# Elements of G-matrix at equilibrium
ind0 = np.where(mask.sum(axis=1) == 0)[0][0]
mu = np.diag(Gmat_coefs[ind0])[:ncoo]

# Second-order derivative of potential at equilibrium
ind2 = np.array(
    [
        np.where((mask.sum(axis=1) == 1) & (deriv_ind[:, icoo] == 2))[0][0]
        for icoo in range(ncoo)
    ]
)
freq = poten_coefs[ind2] * 2

# Linear mapping parameters
lin_a = jnp.sqrt(jnp.sqrt(mu / freq))
lin_b = r0

print("x->r linear mapping parameters 'a':", lin_a)
print("x->r linear mapping parameters 'b':", lin_b)

# Linear mapping function and its derivative
x_to_r_map = lambda x, icoo: lin_a[icoo] * x + lin_b[icoo]
jac_x_to_r_map = lambda x, icoo: np.ones_like(x) * lin_a[icoo]

x->r linear mapping parameters 'a': [0.11245619 0.11245619 0.17845517]
x->r linear mapping parameters 'b': [1.3358387  1.3358387  1.61042427]


Precompute matrix elements of expansion terms in Hermite basis, i.e., $\langle\mathcal{H}_i|r_\lambda^{t_\lambda}|\mathcal{H}_j\rangle$, $\langle\mathcal{H}_i|r_\lambda^{t_\lambda}|\partial_{r_\lambda}\mathcal{H}_j\rangle$, and $\langle\partial_{r_\lambda}\mathcal{H}_i|r_\lambda^{t_\lambda}|\partial_{r_\lambda}\mathcal{H}_j\rangle$, for three internal valence coordinates $\lambda =1..3$.

In [16]:
no_bas = [24] * ncoo
npoints = [80] * ncoo

prim_me = []
prim_dme = []
prim_d2me = []

for icoo in range(ncoo):
    x, w = hermgauss(npoints[icoo])
    w /= np.exp(-(x**2))
    r = x_to_r_map(x, icoo)
    dr = r - r0[icoo]
    r_pow = dr[:, None] ** powers[icoo][None, :]
    psi = hermite(x, np.arange(no_bas[icoo] + 1))
    dpsi = (
        hermite_deriv(x, np.arange(no_bas[icoo] + 1)) / jac_x_to_r_map(x, icoo)[:, None]
    )
    me = jnp.einsum("gi,gt,gj,g->tij", psi, r_pow, psi, w)
    dme = jnp.einsum("gi,gt,gj,g->tij", psi, r_pow, dpsi, w)
    d2me = jnp.einsum("gi,gt,gj,g->tij", dpsi, r_pow, dpsi, w)
    prim_me.append(me)
    prim_dme.append(dme)
    prim_d2me.append(d2me)

Solve contracted problems ...

In [17]:
contr_vec = []

for icoo in range(ncoo):

    bas_ind = [
        np.arange(no_bas[icoo] + 1) if icoo_ == icoo else np.arange(1)
        for icoo_ in range(ncoo)
    ]
    bas_ind, bas_mind = next(generate_prod_ind(bas_ind))

    me = np.prod(
        [
            prim_me[icoo_][
                np.ix_(deriv_mind[icoo_, :], bas_mind[icoo_, :], bas_mind[icoo_, :])
            ]
            for icoo_ in range(ncoo)
        ],
        axis=0,
    )

    d2me = np.prod(
        [
            (
                prim_d2me[icoo_][
                    np.ix_(deriv_mind[icoo_, :], bas_mind[icoo_, :], bas_mind[icoo_, :])
                ]
                if icoo_ == icoo
                else prim_me[icoo_][
                    np.ix_(deriv_mind[icoo_, :], bas_mind[icoo_, :], bas_mind[icoo_, :])
                ]
            )
            for icoo_ in range(ncoo)
        ],
        axis=0,
    )

    vme = me.T @ poten_coefs
    gme = d2me.T @ Gmat_coefs[:, icoo, icoo]
    hme = 0.5 * gme + vme
    e, v = np.linalg.eigh(hme)
    print(icoo, e[0], e - e[0])
    contr_vec.append(v)

0 2361.7892119766816 [     0.           2628.82252588   5170.63202883   7641.24915273
  10067.06594893  12481.61932906  14917.59370122  17400.07732317
  19944.98172078  22561.01501814  25252.33313747  28022.15051195
  30885.0258236   33890.24235674  37139.17960787  40697.11770159
  44626.13760067  49091.75812417  53582.45903274  58870.89057654
  68650.59398697  86262.31051852 114543.63622474 159685.9120161
 238353.59849477]
1 2361.7892120232377 [     0.           2628.82252588   5170.63202882   7641.2491527
  10067.06594885  12481.6193289   14917.59370095  17400.07732277
  19944.98172023  22561.01501743  25252.33313659  28022.15051089
  30885.02582234  33890.24235524  37139.17960607  40697.11769946
  44626.13759814  49091.75812129  53582.45902944  58870.89057254
  68650.59398227  86262.31051334 114543.63621913 159685.91201008
 238353.59848844]
2 2026.3208818639296 [    0.          1214.21472334  2422.60664349  3624.68903371
  4819.88738873  6007.52005336  7186.77178498  8356.65665212
 

Transform primitive matrix elements to the contracted basis

In [18]:
contr_me = [contr_vec[icoo].T @ prim_me[icoo] @ contr_vec[icoo] for icoo in range(ncoo)]
contr_dme = [contr_vec[icoo].T @ prim_dme[icoo] @ contr_vec[icoo] for icoo in range(ncoo)]
contr_d2me = [contr_vec[icoo].T @ prim_d2me[icoo] @ contr_vec[icoo] for icoo in range(ncoo)]

Final solution 

In [19]:
pmax = 12
poly_coefs = np.array([2, 2, 1])

bas_ind = [np.arange(no_bas[icoo] + 1) for icoo in range(ncoo)]
bas_ind, bas_mind = next(
    generate_prod_ind(bas_ind, select=lambda ind: np.sum(ind * poly_coefs) <= pmax)
)
print("polyad contraction number:", pmax)
print("total number of basis products:", len(bas_ind))

me = np.prod(
    [
        contr_me[icoo_][
            np.ix_(deriv_mind[icoo_, :], bas_mind[icoo_, :], bas_mind[icoo_, :])
        ]
        for icoo_ in range(ncoo)
    ],
    axis=0,
)

vme = me.T @ poten_coefs

gme = 0
fac = {}
for icoo in range(ncoo):
    for jcoo in range(ncoo):
        fac[icoo]=1
        fac[jcoo]=-1

        if icoo != jcoo:
            dme = np.prod(
                [
                    (
                        fac[icoo_]*contr_dme[icoo_][
                            np.ix_(
                                deriv_mind[icoo_, :],
                                bas_mind[icoo_, :],
                                bas_mind[icoo_, :],
                            )
                        ]
                        if icoo_ == icoo or icoo_ == jcoo
                        else contr_me[icoo_][
                            np.ix_(
                                deriv_mind[icoo_, :],
                                bas_mind[icoo_, :],
                                bas_mind[icoo_, :],
                            )
                        ]
                    )
                    for icoo_ in range(ncoo)
                ],
                axis=0,
            )
        else:
            dme = np.prod(
                [
                    (
                        contr_d2me[icoo_][
                            np.ix_(
                                deriv_mind[icoo_, :],
                                bas_mind[icoo_, :],
                                bas_mind[icoo_, :],
                            )
                        ]
                        if icoo_ == icoo
                        else contr_me[icoo_][
                            np.ix_(
                                deriv_mind[icoo_, :],
                                bas_mind[icoo_, :],
                                bas_mind[icoo_, :],
                            )
                        ]
                    )
                    for icoo_ in range(ncoo)
                ],
                axis=0,
            )
        gme += dme.T @ Gmat_coefs[:, icoo, jcoo]


hme = 0.5 * gme + vme
e, v = np.linalg.eigh(hme)
print(e[0], e - e[0])

polyad contraction number: 12
total number of basis products: 140
3300.950780627834 [    0.          1182.79513286  2354.55683132  2616.1577043
  2629.46137089  3515.07707792  3781.3055432   3790.53645426
  4664.09639833  4935.44335483  4941.04080678  5154.58801078
  5156.09166356  5245.8819222   5801.26013574  6078.33920966
  6080.76331473  6296.51367276  6296.86394277  6388.94154735
  6926.21183558  7209.65214104  7209.92568363  7427.07757959
  7427.36390327  7522.07520611  7609.14896082  7609.62978955
  7767.4029335   7787.40036812  8038.60422988  8327.6900258
  8330.16766049  8547.15908223  8547.68379987  8645.17325252
  8720.41216468  8720.89806294  8895.53627912  8905.70938492
  9141.43295258  9437.05622077  9443.00869516  9661.82987934
  9662.08261256  9759.34604509  9826.43156788  9826.92582548
 10001.43349757 10001.61861102 10014.59548855 10015.32012922
 10231.49179109 10232.5837198  10237.70234358 10310.90238524
 10538.47615121 10551.29614285 10769.54761773 10770.36189206
 10